# README
1. Edit datasets.metadata_from_dataset
    - Add species_code
2. Bump cache versions
    - config.api.recs.search_recs.params.config
    - load.Load.recs
    - load.Load._metadata

# Notes: call stack for metadata.species.df + xc.metadata + search_recs
  [ebird]
    metadata.species.df (NOTE @cache_to_file_forever: data/metadata/cache/_taxa-v2)
      metadata.species._raw_ebird_df
        # read data/metadata/ebird-*-taxa-species.csv
        metadata.species.add_species_group_cols
          metadata.species._raw_ebird_clements_taxo_df
            ... [taxo]
  [taxo]
    metadata.species._raw_ebird_clements_taxo_df
      # read data/metadata/eBird-Clements-*.csv.gz (taxo)
      # join manual magics (_UNK, _MUL, _NON)
      # join on species_code
      # join on unmatched rows on sci_name
      # join on unmatched rows manually (using a manual table of com_name's)
  [xc]
    sg.xc_meta
      sg.load_xc_meta (NOTE @cache.version)
        datasets.load_xc_meta
          datasets._xc.metadata (NOTE @cache.version)
            datasets._xc._metadata (NOTE @cache.version)
              # read data/xc/metadata/v1/*.pkl (xc_meta)
            # join downloaded_ids
            # join downloaded_page_metadata
            datasets.XCDF
              metadata.species.df
                ... [ebird]
              # add .species FK from metadata.species.df (ebird)
              # join .taxon_order, .taxon_id from metadata.species.df (ebird)
  [search_recs]
    sg.search_recs
      api.recs.get_search_recs
        payloads.df_cache_hybrid (NOTE cache: config.api.recs.search_recs.params.config.version)
          api.recs._compute_search_recs
            sg.xc_meta
              ... [xc]
            api.recs.recs_featurize_pre_rank
              api.recs.recs_featurize_metdata_audio_slice
                api.recs.recs_featurize_metadata
                  datasets.xc_meta_to_raw_recs
                    load.recs (NOTE @cache.version)
                      load.metadata
                        load._metadata (NOTE @cache.version)
                          datasets.metadata_from_dataset
                            metadata.species.df
                              ... [ebird]
                            # join cols from species.*
          payloads.write_mobile_payload
            ... [payloads/mobile]
  [payloads/mobile]
    payloads.write_mobile_payload
      # write data/cache/payloads/*/mobile/

# TODO Simplify the above call stack -- always a time sink when I try to edit it, even for simple additions
- Reorganize around core concepts, aligned with inputs/outputs
    - Inputs (files on disk)
        - ebird (metadata.species.df)
        - taxo  (add_species_group_cols / _raw_ebird_clements_taxo_df)
        - xc    (xc_meta / _xc.metadata)
    - Outputs
        - search_recs     (from ebird + taxo + xc)
        - payloads/mobile (from search_recs)

In [ ]:
from notebooks import *
sg.init(None, skip=[sg_load.load_search_recs])
get_search_recs(write_mobile_payload=True)
sg.init(None, reload=True)

In [ ]:
(metadata.species.df
    [:1].T
)

In [ ]:
(xc.metadata
    [:1].T
)

In [ ]:
# (sg.xc_meta
#     [:1].T
# )

In [ ]:
(sg.search_recs
    [:1].T
)